<a href="https://colab.research.google.com/github/Kewton/kewton.blog.colab/blob/master/%E9%87%8D%E3%81%BF%E3%81%A4%E3%81%8D%E6%9C%80%E5%A4%A7%E4%BA%8C%E9%83%A8%E3%83%9E%E3%83%83%E3%83%81%E3%83%B3%E3%82%B0%E5%95%8F%E9%A1%8CGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# jupyter-dashインストール
!pip install jupyter-dash==0.4.0

# dash_bootstrap_componentsインストール
!pip install dash_bootstrap_components==0.13.1

!pip install dash_cytoscape

     |████████████████████████████████| 7.3 MB 9.8 MB/s 
     |████████████████████████████████| 23.9 MB 1.9 MB/s 
     |████████████████████████████████| 357 kB 67.2 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=691ef171b4795873f25553a9bb201f2df4b21dbf86f9a4e18aa547552c8da6a3
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=cc3c3dc3492fff518e5823206f9cef31374e369f383077654c8ff67fb28693be
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=4047b814191934bb571d9d5778075cd4a4a8d9a4358b2874e73b32bc56335a42
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [2]:
#import dash_html_components as html
from dash import html
import pandas as pd
import base64
import io
import re

def com_replace(_s):
  '''
  dashではidに","（カンマ）は使えないため置換する。
  ※カンマをそのまま置換するとjson形式が崩れるため注意が必要
  ※(1, 1)->1
  
  '''
  pattern = r'(\d{1,4}?), (\d{1,4}?)'
  result = re.sub(pattern, r'\1_\2', str(_s))
  return result 

def parse_content(contents, filename):
    content_type, content_string = contents.split(",")
    decoded = base64.b64decode(content_string)

    try:
        if filename.endswith(".csv"):
            df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))
        elif "xls" in filename:
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div(["ファイルの読み込みでエラーが発生しました"])

    return df

In [3]:
import pandas as pd
import networkx as nx
import dash_cytoscape as cyto
import copy

def addno(_df):
  df_nodedef_0 = _df[_df["属性"]==0].sort_values('名前')
  serial_num_0= pd.RangeIndex(start=1, stop=len(df_nodedef_0.index) + 1, step=1)
  df_nodedef_0["no"]=serial_num_0
  
  df_nodedef_1 = _df[_df["属性"]==1].sort_values('名前')
  serial_num_1 = pd.RangeIndex(start=1, stop=len(df_nodedef_1.index) + 1, step=1)
  df_nodedef_1["no"]=serial_num_1

  return df_nodedef_0, df_nodedef_1

def getval(_edge_weight):
  if "weight" in _edge_weight.keys():
    return _edge_weight["weight"]
  else:
    return 0

def getGraph(df_nodedef, df_edge0to1, df_edge1to0):
  G = nx.Graph()

  df0, df1 = addno(df_nodedef)
  df = pd.concat([df0, df1])

  for index, row in df.iterrows():
    node = (int(row["属性"]), int(row["no"]))
    G.add_node(node)

  for index, row in pd.concat([df_edge0to1, df_edge1to0]).iterrows():
    x_0=df0[df0["名前"]==row["0属性"]]["属性"].iloc[0]
    y_0=df0[df0["名前"]==row["0属性"]]["no"].iloc[0]
    x_1=df1[df1["名前"]==row["1属性"]]["属性"].iloc[0]
    y_1=df1[df1["名前"]==row["1属性"]]["no"].iloc[0]
    node_0=(x_0, y_0)
    node_1=(x_1, y_1)
    G.add_edge(node_0, node_1)
    G[node_0][node_1]["weight"] = getval(G[node_0][node_1]) + row["重み"]
  
  return G, df

def CreateGraphAndMax_weight_matching(_df_nodedef, _df_edge0to1, _df_edge1to0):
    G, df = getGraph(_df_nodedef, _df_edge0to1, _df_edge1to0)
    
    # 重み最大マッチングを解く
    mw = nx.max_weight_matching(G)

    # 扱いやすいように重み最大マッチングの結果のノード名を置換してリストに格納する
    mw_list = []
    for a in mw:
      fromtolist = []
      for b in a:
        fromtolist.append((com_replace(b)))
      mw_list.append(fromtolist)
    
    return G, mw_list, df

def DefaultLayoutOfCytoscape(_element):
  cyto_compo = cyto.Cytoscape(
      id='cytoscape',
      elements=_element,
      layout={
          'name': 'circle','padding': 10
          # 'name': 'grid',"rows":2,"colmuns":5 # グリッドは枝の重みのラベルが重なりやすい
      },
      stylesheet=[{
          'selector': 'node',
          'style': {
              'width': '60px',
              'height': '60px',
              'content': 'data(name)',
              'pie-size': '80%',
              'background-color': 'data(color)',
          }
      }, {
          'selector': 'edge',
          'style': {
              'label': 'data(weight)',
              'width': 'data(weight_width)',
              'curve-style': 'bezier',
              #'target-arrow-shape': 'triangle', # 今回は無向グラフ
              'line-color': 'data(color)',
              'opacity': 0.5
          }
      }, {
          'selector': ':selected',
          'style': {
              'background-color': 'black',
              'line-color': 'black',
              'target-arrow-color': 'black',
              'source-arrow-color': 'black',
              'opacity': 1
          }
      }, {
          'selector': '.faded',
          'style': {
              'opacity': 0.25,
              'text-opacity': 0
          }
      }],
      style={
          'width': '80%',
          'height': '80%',
          'position': 'absolute',
          #'position': 'relative',
          'left': 200,
          'top': 200
      }
  )
  return cyto_compo

def drawCytoscape(_df_nodedef, _df_edge0to1, _df_edge1to0):
  # Step 1. NetworkXのGraphオブジェクトと重み最大マッチング結果リストを取得
  G, mw_list, df = CreateGraphAndMax_weight_matching(_df_nodedef, _df_edge0to1, _df_edge1to0)

  # Step 2. NetworkXのGraphオブジェクト ⇨ Cytoscape用のデータ形式
  cy_data = nx.readwrite.json_graph.cytoscape_data(G)

  # Step 3. Cytoscapeのデータ形式 ⇨ Dash Cytoscapeのデータ形式
  dash_cy_elements = cy_data["elements"]["nodes"] + cy_data["elements"]["edges"]

  # Step 4. 視認性を高めるためにDash Cytoscapeのデータ形式を編集
  aftele = copy.deepcopy(dash_cy_elements)
  for i in range(0, len(dash_cy_elements)):
    ele_list = dash_cy_elements[i]
    for ele_dict in ele_list:
      for dt in ele_list[ele_dict]:
        if "id" in ele_list[ele_dict].keys():
          # nodeの場合
          tp = eval(ele_list[ele_dict]["id"])
          _name = df[(df["属性"]==tp[0]) & (df["no"]==tp[1])]["名前"].iloc[0]
          aftele[i][ele_dict]["id"] = com_replace(ele_list[ele_dict]["id"])
          aftele[i][ele_dict]["value"] = _name
          aftele[i][ele_dict]["name"] = _name+"さん"
          if "(1" in aftele[i][ele_dict]["id"]:
            # 突っ込みの場合
            aftele[i][ele_dict]["color"] = "red"
          else:
            # ボケの場合
            aftele[i][ele_dict]["color"] = "navy"
        else:
          # edgeの場合
          # idを付与する
          aftele[i][ele_dict]["id"] = com_replace(ele_list[ele_dict]["source"]) + "2" + com_replace(ele_list[ele_dict]["target"])
          aftele[i][ele_dict]["source"] = com_replace(ele_list[ele_dict]["source"])
          aftele[i][ele_dict]["target"] = com_replace(ele_list[ele_dict]["target"])
          # 線の太さの差異を強調する
          aftele[i][ele_dict]["weight_width"] = float(aftele[i][ele_dict]["weight"])*1.5
          # デフォルトカラー
          aftele[i][ele_dict]["color"] = "silver"
          for a in mw_list:
            # 重み最大マッチング結果に含まれる場合強調カラー
            if aftele[i][ele_dict]["source"] in set(a) and aftele[i][ele_dict]["target"] in set(a):
              aftele[i][ele_dict]["color"] = "black"

  return DefaultLayoutOfCytoscape(aftele)


In [4]:
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

class JupyterDashBootStrap(JupyterDash):
  def __init__(self, _title, _port):
    super().__init__(__name__, external_stylesheets=self.__external_stylesheets())
    if _title is not None:
      self.title = _title
    self.__setlayout()
    self.__run(_port)
  
  def __setlayout(self):
      self.layout = self.buildbaselayout()
      self.registcalback(self)

  def buildbaselayout(self):
      return
  
  def registcalback(self, app):
      return

  def __run(self, _port):
    if _port is None:
      self.run_server(mode="external")
    else:
      self.run_server(mode="external", port=_port )
  
  def __external_stylesheets(self):
    return [dbc.themes.BOOTSTRAP]

/usr/local/lib/python3.7/dist-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
import pandas as pd

import dash_html_components as html
# import dash_core_components as dcc
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

class MyDashCytoscape(JupyterDashBootStrap):
  _df_nodedef = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
  _df_edge0to1 = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
  _df_edge1to0 = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
  
  def buildbaselayout(self):
    upload_style = {
        "width": "50%",
        "height": "120px",
        "lineHeight": "60px",
        "borderWidth": "1px",
        "borderStyle": "dashed",
        "borderRadius": "5px",
        "textAlign": "center",
        "margin": "10px",
        "margin": "3% auto",
    }
    _up1 = dcc.Upload(
                id="nodedef",
                children=html.Div(["ノード属性", html.A("のcsvファイル")]),
                style=upload_style,
            )
    _up2 = dcc.Upload(
                id="edge0to1",
                children=html.Div(["エッジリスト_0to1", html.A("のcsvファイル")]),
                style=upload_style,
            )
    _up3 = dcc.Upload(
                id="edge1to0",
                children=html.Div(["エッジリスト_1to0", html.A("のcsvファイル")]),
                style=upload_style,
            )

    row = html.Div(
        [
            dbc.Row(
                [
                    dbc.Col(_up1),
                    dbc.Col(_up2),
                    dbc.Col(_up3),
                ],
                align="start",
            ),
            dbc.Button("Matching!", id="my_button", className="me-2", n_clicks=0),
            html.Div(id="my_div1", children=[]),
            html.Div(id="my_div2", children=[]),
            html.Div(id="my_div3", children=[]),
            html.Div(id="my_div4", children=[]),
            html.Div(id="my_div", children=[]),
        ]
    )
    return row
    
  def registcalback(self, app):
    @app.callback(
        [
            Output("my_div1", "children"),
        ],
        [Input("nodedef", "contents")],
        [State("nodedef", "filename")],
        prevent_initial_call=True,
    )
    def upload_nodedef(contents, filename):
        print("■ upload_nodedef")
        self._df_nodedef = parse_content(contents, filename)
        print(self._df_nodedef.info())
        return [f"ノード属性ファイル：{filename}"]

    @app.callback(
        [
            Output("my_div2", "children"),
        ],
        [Input("edge0to1", "contents")],
        [State("edge0to1", "filename")],
        prevent_initial_call=True,
    )
    def upload_edge0to1(contents, filename):
        print("■ upload_edge0to1")
        self._df_edge0to1 = parse_content(contents, filename)
        print(self._df_edge0to1.info())
        return [f"エッジリスト_0to1ファイル：{filename}"]

    @app.callback(
        [
            Output("my_div3", "children"),
        ],
        [Input("edge1to0", "contents")],
        [State("edge1to0", "filename"),],
        prevent_initial_call=True,
    )
    def upload_edge1to0(contents, filename):
        print("■ upload_edge1to0")
        self._df_edge1to0 = parse_content(contents, filename)
        print(self._df_edge1to0.info())
        return [f"エッジリスト_1to0ファイル：{filename}"]

    @app.callback(
        [
            Output("my_div4", "children"),
            Output("my_div", "children"),
        ],
        [Input("my_button", "n_clicks")],
        prevent_initial_call=True,
    )
    def drawmatching(n_clicks):
        print("drawmatching")
        try:
            new_layout=drawCytoscape(self._df_nodedef, self._df_edge0to1, self._df_edge1to0)
            return "success!", [new_layout]
        except Exception as e:
            print("--- Error ---")
            print(e)
            return "File format is incorrect", []


In [6]:
MyDashCytoscape("*** matching ***",None)

Dash app running on:


<IPython.core.display.Javascript object>